In [16]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import numpy as np

import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

In [17]:
def default_transform(df):
    return df

def read_chunk(name, chunk_id, opts=csv.ConvertOptions(), transform=default_transform, extra=1):
    chunk=None
    chunksize = extra*10 ** 7
    with csv.open_csv(
        name,
        convert_options = opts,
        read_options=csv.ReadOptions(
            use_threads=True,
            block_size=chunksize
        )) as reader:

        i=0
        for next_chunk in reader:
            if next_chunk is None:
                break
            chunk=next_chunk
            if i == chunk_id:
                return transform(chunk.to_pandas())
            i+=1
            
    return None

def prep_NF_UQ(df):
    features_to_remove = ['L4_SRC_PORT', 'L4_DST_PORT', 'Attack', 'L4_SRC_PORT', 'L4_DST_PORT']
    df.drop(columns=features_to_remove, axis=1, inplace=True) 
    df['mask'] = 1
    return df

def prep_ddos(df):
    features_to_remove = ['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Timestamp']
    df.drop(columns=features_to_remove, axis=1, inplace=True) 
    
    df = df.rename(columns=
                   {' Protocol': 'PROTOCOL', 
                   ' Total Fwd Packets': 'IN_PKTS', 
                   ' Total Backward Packets': 'OUT_PKTS',
                   'Total Length of Fwd Packets': 'IN_BYTES',
                   ' Total Length of Bwd Packets': 'OUT_BYTES',
                   'Fwd IAT Total': 'DURATION_IN',
                   'Bwd IAT Total': 'DURATION_OUT',
                   ' Label': 'Attack',
                   ' Flow Duration': 'FLOW_DURATION_MILLISECONDS',
                   }, errors="raise")
    
    df['PROTOCOL'] = df['PROTOCOL'].astype(np.int64)
    df['IN_PKTS'] = df['IN_PKTS'].astype(np.int64)
    df['OUT_PKTS'] = df['OUT_PKTS'].astype(np.int64)
    df['IN_BYTES'] = df['IN_BYTES'].astype(np.int64)
    df['OUT_BYTES'] = df['OUT_BYTES'].astype(np.int64)
    df['DURATION_IN']= (df['DURATION_IN']/1000000).astype(np.int64)
    df['DURATION_OUT']= (df['DURATION_OUT']/1000000).astype(np.int64)
    df['Label'] = 1
    df['mask'] = 2
    return df

In [18]:
cols = prep_NF_UQ(pd.read_parquet('NF-UNSW-NB15-V2.parquet')).columns
cols = np.unique(np.concatenate((cols, prep_ddos(read_chunk('01-12/DrDoS_UDP.csv', chunk_id=0)).columns), axis=0))

def append(dataset, df):
    df[cols[np.invert(np.isin(cols, df.columns))]] = 0
    return dataset.append(df, ignore_index=True)

In [19]:
dataset = pd.DataFrame(columns=cols)

In [20]:
from os import listdir
from os.path import isfile, join

files = ['./01-12','./03-11']
for file in files:
    for f in listdir(file):
        if f[0] == '.':
            continue
        filename=file+"/"+f
        dataset = append(dataset, prep_ddos(read_chunk(filename, 0, extra=1)))

/tmp/ipykernel_139870/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_139870/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_139870/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_139870/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return dataset.append(df, ignore_index=True)
/tmp/ipykernel_139870/1702868408.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

In [21]:
dataset

,ACK Flag Count,Active Max,Active Min,Active Std,Average Packet Size,Avg Bwd Segment Size,Avg Fwd Segment Size,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Header Length,...,SERVER_TCP_FLAGS,SHORTEST_FLOW_PKT,SRC_TO_DST_AVG_THROUGHPUT,SRC_TO_DST_SECOND_BYTES,SimillarHTTP,Subflow Fwd Packets,TCP_FLAGS,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,mask
0,0,67834732.0,4024278.0,2.853681e+07,0.00,0.0,0.0,0,0,0,...,0,0,0,0,0,60959,0,0,0,2
1,0,0.0,0.0,0.000000e+00,2208.00,0.0,1472.0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
2,0,0.0,0.0,0.000000e+00,2208.00,0.0,1472.0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
3,0,0.0,0.0,0.000000e+00,2208.00,0.0,1472.0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
4,0,0.0,0.0,0.000000e+00,2136.00,0.0,1424.0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408523,0,0.0,0.0,0.000000e+00,46.75,47.0,31.0,0,0,40,...,0,0,0,0,0,2,0,0,0,2
408524,0,0.0,0.0,0.000000e+00,46.75,47.0,31.0,0,0,40,...,0,0,0,0,0,2,0,0,0,2
408525,0,0.0,0.0,0.000000e+00,39.25,41.0,25.0,0,0,40,...,0,0,0,0,0,2,0,0,0,2
408526,0,0.0,0.0,0.000000e+00,64.25,82.0,31.0,0,0,40,...,0,0,0,0,0,2,0,0,0,2


In [22]:
def get_data(df, scaler=None, useScaler=False):
    Y = df.Label
    df = df.drop(['Label'], axis=1)

    df['Attack'] = df['Attack'].astype("string")

    X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3,stratify=df.Attack, random_state=42)
    del df
    del Y

    attack_train = X_train['Attack']
    X_train = X_train.drop('Attack', axis=1)
    attack_test = X_test['Attack']
    X_test = X_test.drop('Attack', axis=1)

    X_train = X_train.drop(['SimillarHTTP'], axis=1)
    X_train[X_train['Flow Bytes/s'] == np.inf] = 0
    X_train[X_train[' Flow Packets/s'] == np.inf] = 0

    X_test = X_test.drop(['SimillarHTTP'], axis=1)
    X_test[X_test['Flow Bytes/s'] == np.inf] = 0
    X_test[X_test[' Flow Packets/s'] == np.inf] = 0

    
    if useScaler:
        X_train = scaler.transform(X_train)
    else:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test, attack_train, attack_test, scaler

In [23]:
dataset.to_csv('cic-ddos2019.csv')

In [14]:
X_train.shape

(285969, 111)

In [15]:
y_train.shape

(285969,)